In [6]:
#spacy english en_core_web_trf

#Librerias a usar

#from bert_serving.client import BertClient
import spacy
from collections import Counter
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

#import torch; print(torch.__version__)
import torch
from keybert import KeyBERT



[nltk_data] Downloading package punkt to
[nltk_data]     /home/alvaromontero/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alvaromontero/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
#Cargar libros
Book_list=["/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/divine_comedy.txt",'/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Oddysey.txt'
           ,'/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Illiad.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Great_Gatsby.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Tom_Sawyer.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Wuthering_Heights.txt']
#Book_list=[r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\divine_comedy.txt",r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Oddysey.txt",r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Illiad.txt",
#                r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Great_Gatsby.txt",r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Tom_Sawyer.txt",r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Wuthering_Heights.txt"]

Books_raw_file=[]
for i in range(len(Book_list)):
    with open(Book_list[i],'r',encoding='utf-8')as f:
        file=f.read()
        Books_raw_file.append(file)
        
        


PREPROCESAMIENTO


In [8]:
#Funciones

def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-z\s]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [9]:
#Vectorizacion tfidf
def vectorizar_tfidf(corpus):
    # TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=(4,4))
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    # palabras únicas
    feature_names_tfidf = vectorizer.get_feature_names_out()
    vocabulary = vectorizer.get_feature_names_out()
    print("Tamaño de Palabras únicas:", len(feature_names_tfidf))
    return tfidf_matrix,vocabulary

In [10]:
#Quitar inicio de texto
deleted_lines_list=[134,336,1997,32,459,7]

for i,book in enumerate(Books_raw_file):
    lines = book.splitlines()  # Split the book text into lines
    modified_lines = lines[deleted_lines_list[i]:]  # Keep lines starting from the specified index
    modified_book = "\n".join(modified_lines)  # Join the remaining lines back into a string
    Books_raw_file[i]=modified_book


In [11]:
for i,book in enumerate(Books_raw_file):
    Books_raw_file[i]=book.lower()
    
print(Books_raw_file[0][:200])

canto i


in the midway of this our mortal life,
i found me in a gloomy wood, astray
gone from the path direct: and e’en to tell
it were no easy task, how savage wild
that forest, how robust and rough


Eliminacion de caracteres no importantes


In [12]:
from copy import deepcopy
books_clean_files=[]
books_clean_stops_file=[]
for i,book in enumerate(Books_raw_file):
    Text_aux=eliminar_etiquetas(book)
    Text_aux=remover_apostrofes(Text_aux)
    Text_aux=dobles_espacios(Text_aux)
    Text_aux=remover_especiales(Text_aux)
    
    books_clean_stops_file.append(deepcopy(Text_aux))#Por si el modelo no necesita quitar stopwords
    
    Text_aux=remove_stopwords(Text_aux)
    books_clean_files.append(Text_aux)#Texto sin stopwords
    print(Text_aux[:45]+'\n')

canto midway mortal life found gloomy wood as

book gods council minerva visit ithaca challe

iliad book argument contention achilles agame

younger vulnerable years father gave advice t

chapter tom answer tom answer gone boy wonder

chapter returned visit landlord solitary neig



Extracción de oraciones clave

1) TF-IDF

In [13]:
#TF-IDF NLTK

for book in books_clean_files:#Iteramos sobre los libros para obtener lo mas representativo de cada texto
    #tokens=word_tokenize(book)
    #postags=pos_tag(tokens)
    #stemmer=PorterStemmer()
    #stemmed_tokens=[stemmer.stem(token) for token in tokens]
    
    
    vectorizer = TfidfVectorizer(ngram_range=(4,4))#Vectorizador con 4 gramas
    tfidf_matrix = vectorizer.fit_transform([book])
    feature_names_tfidf = vectorizer.get_feature_names_out()
    vocabulary = vectorizer.get_feature_names_out()#Obtenemos el bocabulario
    print("Tamaño de Palabras únicas:", len(feature_names_tfidf))
    frecuencia_total = Counter()
    for ngrama in vocabulary:#Contamos las frases mas comunes
    # Obtiene el índice de la palabra en el vocabulario
        indice = vectorizer.vocabulary_[ngrama]
        # Suma las frecuencias de esa palabra en todas las frases
        frecuencia_total[ngrama] = tfidf_matrix[:, indice].sum()
    frases_mas_frecuentes = frecuencia_total.most_common(5)
    for i in range(5):
        print(f"Frase: {frases_mas_frecuentes[i][0]}, Frecuencia: {frases_mas_frecuentes[i][1]}")#El decimal es que tanto contribuyen al "Corpus"




Tamaño de Palabras únicas: 58074
Frase: little ye shall see, Frecuencia: 0.008296684166811896
Frase: phaeton ill knew guide, Frecuencia: 0.008296684166811896
Frase: power one ask thou, Frecuencia: 0.008296684166811896
Frase: speak whence ye stand, Frecuencia: 0.008296684166811896
Frase: tell thou know st, Frecuencia: 0.008296684166811896
Tamaño de Palabras únicas: 54649
Frase: child morning rosy fingered, Frecuencia: 0.07648926028989292
Frase: morning rosy fingered dawn, Frecuencia: 0.07648926028989292
Frase: rosy fingered dawn appeared, Frecuencia: 0.07648926028989292
Frase: soon enough eat drink, Frecuencia: 0.04428325595730643
Frase: ulysses noble son laertes, Frecuencia: 0.04025750541573312
Tamaño de Palabras únicas: 96575
Frase: arm wield avenging bolt, Frecuencia: 0.009506006921700063
Frase: arms resound broad buckler, Frecuencia: 0.009506006921700063
Frase: conquest mighty jove decide, Frecuencia: 0.009506006921700063
Frase: contend juno suit heavenly, Frecuencia: 0.009506006921

  2.- BERT

In [17]:
#Extracción de frases clave usando BERT
key_model=KeyBERT()
for book in books_clean_stops_file:
    key_phrases=key_model.extract_keywords(book,keyphrase_ngram_range=(3,3),nr_candidates=5)
    print(key_phrases)

/home/alvaromontero/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[('mourn desolated realm', 0.5217), ('gone inwardly mourns', 0.504), ('death view dwell', 0.5027), ('misery drear wilderness', 0.4968), ('despairing shrieks spirits', 0.4877)]
[('gods settled ithaca', 0.6837), ('gods people ithaca', 0.6727), ('immortal gods ithaca', 0.6609), ('gods did ulysses', 0.6577), ('gods mean ulysses', 0.6269)]
[('greeks achilles troops', 0.6745), ('achilles agamemnon war', 0.671), ('achilles book argument', 0.6485), ('achilles pursues trojans', 0.6463), ('iliad destiny achilles', 0.6449)]
[('soft mindedness suspecting', 0.4942), ('sleep preoccupation hostile', 0.4296), ('college unjustly accused', 0.4243), ('taking scepticism granted', 0.4202), ('unjustly accused politician', 0.4148)]
[('chapter xxiv tom', 0.4761), ('chapter viii tom', 0.4744), ('chapter xxii tom', 0.4681), ('tom contemplated boy', 0.4634), ('boy tom thought', 0.4606)]
[('counselled mr heathcliff', 0.5678), ('grange mr heathcliff', 0.5658), ('grange said heathcliff', 0.5649), ('politeness mr he

3.-RAKE